<a href="https://colab.research.google.com/github/NerdyCodeEngineer/opponent_Analysis_ODI/blob/main/ODI_opponent_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load dataset (Replace 'odi_extended_dataset.csv' with actual file)
data = pd.read_csv('/content/detailed_player_data.csv')

# Handle missing values
data = data.dropna(subset=['match_outcome'])  # Remove rows with NaN in target column

data['match_outcome'] = data['match_outcome'].map({'Win': 1, 'Loss': 0, 'No Result': 2})

# Check if 'match_outcome' has any values after mapping
if not data['match_outcome'].empty:
    # Reset index to ensure it starts from 0
    data.reset_index(drop=True, inplace=True)

    # Fill remaining NaNs if any, with the mode
    # If .mode() result is empty, fill with a default value (e.g., 1 for "Win")
    if not data['match_outcome'].mode().empty:
        data['match_outcome'].fillna(data['match_outcome'].mode()[0], inplace=True)
    else:
        # Handle the case where the mode Series is empty
        data['match_outcome'].fillna(1, inplace=True)

# Encode categorical data
encoder = LabelEncoder()
data['team'] = encoder.fit_transform(data['team'])
data['opposition_team'] = encoder.fit_transform(data['opposition_team'])
data['venue'] = encoder.fit_transform(data['venue'])

# Selecting relevant features
features = ['team', 'opposition_team', 'venue', 'runs', 'balls_faced', 'fours', 'sixes', 'wickets',
            'overs_bowled', 'balls_bowled', 'runs_conceded', 'catches', 'run_outs', 'maiden', 'stumps',
            'strike_rate', 'economy', 'fantasy_points']
target = 'match_outcome'

# Define input and output
X = data[features]
y = data[target]

# Standardize numerical features
scaler = StandardScaler()
X[features[3:]] = scaler.fit_transform(X[features[3:]].values)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Decision Tree Model
clf = DecisionTreeClassifier(max_depth=5, random_state=42)
clf.fit(X_train, y_train)
y_pred_tree = clf.predict(X_test)
print("Decision Tree Accuracy:", accuracy_score(y_test, y_pred_tree))
print(classification_report(y_test, y_pred_tree))

# Neural Network Model (MLP)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # 3 outputs for Win, Loss, No Result
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_test, y_test))

# Predictions
nn_preds = model.predict(X_test)
nn_preds = np.argmax(nn_preds, axis=1)
print("Neural Network Accuracy:", accuracy_score(y_test, nn_preds))
print(classification_report(y_test, nn_preds))

# User Input for Strategy Analysis
num_players = int(input("Enter the number of player comparisons: "))
player_opponent_pairs = []

for i in range(num_players):
    player_name = input(f"Enter player {i+1}'s name from your team: ")
    opponent_name = input(f"Enter opponent {i+1}'s name: ")
    player_opponent_pairs.append((player_name, opponent_name))

# Generate strategy and winning probability
for player_name, opponent_name in player_opponent_pairs:
    player_stats = data[data['team'] == encoder.transform([player_name])[0]]
    opponent_stats = data[data['opposition_team'] == encoder.transform([opponent_name])[0]]

    if not player_stats.empty and not opponent_stats.empty:
        best_scenario = player_stats[['runs', 'balls_faced', 'strike_rate', 'economy']].mean() - \
                        opponent_stats[['runs_conceded', 'balls_bowled', 'strike_rate', 'economy']].mean()

        print(f"\nStrategy for {player_name} against {opponent_name}:")
        print(best_scenario)

        if best_scenario['strike_rate'] > 100:
            print(f"{player_name} should play an aggressive role against {opponent_name}.")
        elif best_scenario['economy'] < 6:
            print(f"{opponent_name} is an economical bowler, {player_name} should focus on rotating strike.")
        else:
            print(f"Balanced approach recommended for {player_name} against {opponent_name}.")

        # Predict winning probability
        player_input = np.array(player_stats[features].mean()).reshape(1, -1)
        win_prob = model.predict(player_input)
        win_prob = win_prob[0][1] * 100  # Probability of Win (Index 1)
        print(f"Winning Probability against {opponent_name}: {win_prob:.2f}%")
    else:
        print(f"Data not found for {player_name} or {opponent_name} in the dataset.")
